In [ ]:
#!/usr/bin/env python3

# Global imports
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, Collections as col
import GlobalSets.c2l_functions as c2l

# Import librarys
from pymongo import collection, errors
import bson

def local2cloud():
    while(True):
        for collection in col.Collections:
                c2l.downloadBase(database=db.dbBuffer, collection=collection['name'])

if __name__ == '__main__':
    try:
        local2cloud()
    except Exception:
        pass


In [ ]:
#!/usr/bin/env python3

# Global imports
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, Collections as col

# Import librarys
from pymongo import collection, errors
from bson import ObjectId
import bson, json, datetime

def documentHandler(x):
    if isinstance(x, datetime.datetime):
        return x.isoformat()
    elif isinstance(x, bson.objectid.ObjectId):
        return str(x)
    else:
        raise TypeError(x)

def defDiffElements(collection: str):
    pipeline = [
        {
            '$project': {
                '_id': 1
            }
        }
    ]
    Local = json.loads(json.dumps(list(MongoClient.LocalClient[db.dbDashboard][collection].aggregate(pipeline=pipeline)),default=documentHandler))
    Cloud = json.loads(json.dumps(list(MongoClient.CloudClient[db.dbDashboard][collection].aggregate(pipeline=pipeline)),default=documentHandler))

    idLocal = [i['_id'] for i in Local]
    idCloud = [i['_id'] for i in Cloud]

    for i in idLocal:
        for j in idCloud:
            if i == j:
                idLocal.remove(i)
                idCloud.remove(j)

    for id in idCloud:
        try:
            if MongoClient.CloudClient.is_primary:
                MongoClient.CloudClient[db.dbDashboard][collection].delete_one({'_id': ObjectId(id)})
        except Exception as e:
            eStr = str(e)
            print(eStr) 

    for id in idLocal:
        try:
            document =  MongoClient.LocalClient[db.dbDashboard][collection].find_one({'_id': ObjectId(id)})
            for upTry in range(0,5):
                if MongoClient.CloudClient.is_primary and MongoClient.CloudClient[db.dbDashboard][collection].insert_one(document).acknowledged:
                    break
        except Exception as e:
            eStr = str(e)
            print(eStr)  

    
defDiffElements('Battery_actual')

In [16]:
# Global imports
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, DashboardCollections as col
import GlobalSets.l2c_functions as l2c

# Import librarys
from pymongo import collection, errors
import bson, datetime, json, bson

def documentHandler(x):
    if isinstance(x, datetime.datetime):
        return x.isoformat()
    elif isinstance(x, bson.objectid.ObjectId):
        return str(x)
    else:
        raise TypeError(x)


pipelineCol = MongoClient.LocalClient[db.dbDashboard]['pipelines']

def newPipeline(name: str, database: str, collection: str, pipeline: dict):
    document = {
        'name' : name,
        'database': database,
        'collection': collection,
        'pipeline' : pipeline
    }
    try:
        if pipelineCol.insert_one(document=document).acknowledged:
            print('Added')
    except Exception as e:
        eStr = str(e)
        print(eStr)

def executePipelines():
    pipelines = json.loads(json.dumps(list(pipelineCol.find({})),default=documentHandler))
    for pipeline in pipelines:
        MongoClient.LocalClient[pipeline['database']][pipeline['collection']].aggregate(pipeline=pipeline['pipeline'])

executePipelines()
            

ImportError: cannot import name 'DashboardCollections' from 'GlobalSets.Mongo' (/media/andre/98201a4b-9b4b-4da7-aeaa-59369694a00b/GitHub/UGV-backend/DatabaseSync/GlobalSets/Mongo.py)

In [22]:
from runPipelines import newPipeline

pipeline = [
    {
        '$sort': {
            'dateTime': -1
        }
    }, {
        '$limit': 1
    }, {
        '$out': {
            'db': 'CeDRI_UGV_dashboard', 
            'coll': 'Battery_actual'
        }
    }
]

newPipeline("Get actual battery", "CeDRI_UGV_buffer", "Battery_Data", pipeline=pipeline)
    

Added
